# নমুনা ০৬: বুদ্ধিমান মডেল রাউটার - টুল হিসেবে মডেল

এই নোটবুকটি একটি বুদ্ধিমান রাউটিং সিস্টেম প্রদর্শন করে যা বিভিন্ন ধরণের কাজের জন্য স্বয়ংক্রিয়ভাবে সবচেয়ে উপযুক্ত AI মডেল নির্বাচন করে। এটি Microsoft Foundry Local-এর সাথে "টুল হিসেবে মডেল" ধারণা তুলে ধরে।

## সংক্ষিপ্ত বিবরণ

মডেল রাউটার বুদ্ধিমান কাজ শ্রেণীবিভাগ এবং মডেল নির্বাচন বাস্তবায়ন করে:

- 🎯 **কাজ শ্রেণীবিভাগ**: ব্যবহারকারীর প্রশ্ন স্বয়ংক্রিয়ভাবে শ্রেণীবদ্ধ করে
- 🔧 **মডেল নির্বাচন**: কাজের সাথে বিশেষায়িত মডেলগুলির মিল ঘটায়
- ⚙️ **ডাইনামিক কনফিগারেশন**: পরিবেশ-ভিত্তিক টুল রেজিস্ট্রি
- 📊 **স্বাস্থ্য পর্যবেক্ষণ**: পরিষেবার অবস্থা এবং মডেলের প্রাপ্যতা
- 🚀 **প্রোডাকশন রেডি**: এন্টারপ্রাইজ-গ্রেড রাউটিং এবং ত্রুটি পরিচালনা


## আর্কিটেকচার সংক্ষিপ্ত বিবরণ

```
User Query → Task Classifier → Model Selector → Specialized Model → Response
     ↓              ↓               ↓              ↓
"Fix this code" → Code Task → qwen2.5-7b → Code Model → Fixed Code
"Why is...?"   → Reasoning  → deepseek-r1 → Reasoning → Analysis
"Write story"  → Creative   → phi-4-mini  → Creative → Story
```


## প্রয়োজনীয়তা এবং সেটআপ

সেরা রাউটিং নিশ্চিত করতে, আপনার একাধিক মডেল চালু থাকা উচিত। চলুন পরিবেশ সেটআপ করি:


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## লাইব্রেরি আমদানি এবং কনফিগারেশন


In [ ]:
import os
import sys
import re
import json
import subprocess
import time
from typing import Dict, Any, Optional, List
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## মডেল রাউটার ক্লাস

প্রধান রাউটার যা বুদ্ধিমান মডেল নির্বাচন পরিচালনা করে:


In [ ]:
class ModelRouter:
    """Intelligent model router that selects appropriate models for different task types."""
    
    def __init__(self):
        self.client = None
        self.base_url = None
        self.tools = self._load_tool_registry()
        self._initialize_client()
    
    def _load_tool_registry(self) -> Dict[str, Dict[str, Any]]:
        """Load tool registry from environment or use defaults."""
        default_tools = {
            "general": {
                "model": "Phi-4-mini-instruct-cuda-gpu",
                "notes": "Fast general-purpose chat and Q&A",
                "temperature": 0.7
            },
            "reasoning": {
                "model": "deepseek-r1-distill-qwen-7b-cuda-gpu",
                "notes": "Step-by-step analysis and logical reasoning",
                "temperature": 0.3
            },
            "code": {
                "model": "qwen2.5-7b-instruct-cuda-gpu",
                "notes": "Code generation, debugging, and technical tasks",
                "temperature": 0.2
            },
            "creative": {
                "model": "Phi-4-mini-instruct-cuda-gpu",
                "notes": "Creative writing and storytelling",
                "temperature": 0.9
            }
        }
        
        # Override with environment variables
        if os.environ.get("GENERAL_MODEL"):
            default_tools["general"]["model"] = os.environ["GENERAL_MODEL"]
        if os.environ.get("REASONING_MODEL"):
            default_tools["reasoning"]["model"] = os.environ["REASONING_MODEL"]
        if os.environ.get("CODE_MODEL"):
            default_tools["code"]["model"] = os.environ["CODE_MODEL"]
        if os.environ.get("CREATIVE_MODEL"):
            default_tools["creative"]["model"] = os.environ["CREATIVE_MODEL"]
        
        # Check for complete JSON override
        tools_env = os.environ.get("TOOL_REGISTRY")
        if tools_env:
            try:
                custom_tools = json.loads(tools_env)
                return custom_tools
            except json.JSONDecodeError:
                print("⚠️ Invalid TOOL_REGISTRY JSON, using defaults")
        
        return default_tools
    
    def _discover_base_url(self, default: str = "http://localhost:8000") -> str:
        """Discover Foundry Local service URL."""
        env_url = os.environ.get("BASE_URL")
        if env_url:
            return env_url
        
        try:
            # Try to get URL from Foundry CLI
            result = subprocess.run(
                ["foundry", "service", "status"],
                capture_output=True, text=True, timeout=3
            )
            if result.returncode == 0:
                match = re.search(r"https?://[\w\.-]+(?::\d+)?", result.stdout)
                if match:
                    return match.group(0)
        except (subprocess.TimeoutExpired, subprocess.CalledProcessError, FileNotFoundError):
            pass
        
        return default
    
    def _initialize_client(self):
        """Initialize OpenAI client with Foundry Local or fallback configuration."""
        if FOUNDRY_SDK_AVAILABLE:
            try:
                # Try to use any available model for client initialization
                first_model = next(iter(self.tools.values()))["model"]
                print(f"🔄 Initializing Foundry Local SDK with model: {first_model}...")
                manager = FoundryLocalManager(first_model)
                
                self.client = OpenAI(
                    base_url=manager.endpoint,
                    api_key=manager.api_key
                )
                self.base_url = manager.endpoint
                print(f"✅ Foundry Local SDK initialized")
                return
            except Exception as e:
                print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
        
        # Fallback to manual configuration
        self.base_url = self._discover_base_url()
        api_key = os.environ.get("API_KEY", "")
        
        self.client = OpenAI(
            base_url=f"{self.base_url}/v1",
            api_key=api_key
        )
        print(f"🔧 Manual configuration initialized at {self.base_url}")
    
    def check_service_health(self) -> Dict[str, Any]:
        """Check Foundry Local service health and available models."""
        try:
            # Try to list models
            models_response = self.client.models.list()
            available_models = [model.id for model in models_response.data]
            
            # Check which configured models are available
            configured_models = [tool["model"] for tool in self.tools.values()]
            available_configured = [m for m in configured_models if m in available_models]
            
            return {
                "status": "healthy",
                "base_url": self.base_url,
                "available_models": available_models,
                "configured_models": configured_models,
                "available_configured": available_configured,
                "tools_configured": list(self.tools.keys())
            }
        except Exception as e:
            return {
                "status": "error",
                "base_url": self.base_url,
                "error": str(e)
            }
    
    def select_tool(self, user_query: str) -> str:
        """Select the most appropriate tool based on the user query."""
        query_lower = user_query.lower()
        
        # Code-related keywords
        code_keywords = [
            "code", "python", "function", "class", "method", "bug", "debug", 
            "programming", "script", "algorithm", "implementation", "refactor",
            "syntax", "compile", "error", "exception", "variable", "loop"
        ]
        if any(keyword in query_lower for keyword in code_keywords):
            return "code"
        
        # Reasoning keywords
        reasoning_keywords = [
            "why", "how", "explain", "step-by-step", "reason", "analyze", 
            "think", "logic", "because", "cause", "compare", "evaluate",
            "pros and cons", "advantage", "disadvantage", "benefit", "drawback"
        ]
        if any(keyword in query_lower for keyword in reasoning_keywords):
            return "reasoning"
        
        # Creative keywords
        creative_keywords = [
            "story", "poem", "creative", "imagine", "write", "tale", 
            "narrative", "fiction", "character", "plot", "novel", "poetry",
            "song", "lyrics", "dialogue", "script"
        ]
        if any(keyword in query_lower for keyword in creative_keywords):
            return "creative"
        
        # Default to general
        return "general"
    
    def chat(self, model: str, content: str, max_tokens: int = 300, temperature: Optional[float] = None) -> str:
        """Send chat completion request to the specified model."""
        try:
            params = {
                "model": model,
                "messages": [{"role": "user", "content": content}],
                "max_tokens": max_tokens
            }
            
            if temperature is not None:
                params["temperature"] = temperature
            
            response = self.client.chat.completions.create(**params)
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating response with model {model}: {str(e)}"
    
    def route_and_run(self, prompt: str) -> Dict[str, Any]:
        """Route the prompt to the appropriate model and generate response."""
        tool_key = self.select_tool(prompt)
        tool_config = self.tools[tool_key]
        model = tool_config["model"]
        temperature = tool_config.get("temperature", 0.7)
        
        start_time = time.time()
        answer = self.chat(
            model=model, 
            content=prompt, 
            max_tokens=400, 
            temperature=temperature
        )
        end_time = time.time()
        
        return {
            "tool": tool_key,
            "model": model,
            "tool_description": tool_config["notes"],
            "temperature": temperature,
            "processing_time": end_time - start_time,
            "answer": answer,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }

# Initialize the router
print("Initializing Model Router...")
print("=" * 50)
router = ModelRouter()
print("=" * 50)
print("✅ Model Router initialized!")

## সার্ভিস স্বাস্থ্য পরীক্ষা

চলুন আমাদের Foundry Local সার্ভিসের স্বাস্থ্য পরীক্ষা করি এবং দেখি কোন মডেলগুলো উপলব্ধ আছে:


In [ ]:
# Check service health and available models
health = router.check_service_health()

print("🏥 **Service Health Check**")
print("=" * 50)
print(f"🚦 Status: {health['status']}")
print(f"🔗 Base URL: {health['base_url']}")

if health['status'] == 'healthy':
    print(f"\n📋 **Available Models:**")
    for i, model in enumerate(health['available_models'], 1):
        print(f"   {i}. {model}")
    
    print(f"\n🎯 **Configured Models:**")
    for tool, config in router.tools.items():
        model = config['model']
        status = "✅ Available" if model in health['available_models'] else "❌ Not Available"
        print(f"   {tool.title()}: {model} - {status}")
    
    available_tools = len(health['available_configured'])
    total_tools = len(health['configured_models'])
    print(f"\n📊 **Tools Ready:** {available_tools}/{total_tools}")
    
    if available_tools < total_tools:
        missing_models = set(health['configured_models']) - set(health['available_configured'])
        print(f"⚠️ **Missing Models:** {', '.join(missing_models)}")
        print("💡 To start missing models, run: foundry model run <model-name>")
else:
    print(f"❌ **Error:** {health.get('error', 'Unknown error')}")
    print("\n🔧 **Troubleshooting:**")
    print("1. Ensure Foundry Local is running: foundry service status")
    print("2. Start a model: foundry model run phi-4-mini")
    print("3. Check the endpoint URL is correct")

## শ্রেণীবিন্যাস পরীক্ষা

চলুন রাউটারের বিভিন্ন ধরণের প্রশ্ন শ্রেণীবিন্যাস করার ক্ষমতা পরীক্ষা করি:


In [ ]:
def test_classification(test_queries: List[str]):
    """Test the classification system with various queries."""
    print("🔍 **Classification Testing**")
    print("=" * 60)
    
    for i, query in enumerate(test_queries, 1):
        tool = router.select_tool(query)
        tool_config = router.tools[tool]
        print(f"\n{i}. **Query:** {query}")
        print(f"   🎯 **Classified as:** {tool.title()}")
        print(f"   🤖 **Model:** {tool_config['model']}")
        print(f"   🌡️ **Temperature:** {tool_config['temperature']}")
        print(f"   📝 **Purpose:** {tool_config['notes']}")

# Test classification with diverse queries
test_queries = [
    # General queries
    "What is artificial intelligence?",
    "Tell me about Microsoft Foundry Local",
    
    # Code queries
    "Write a Python function to sort a list",
    "Fix this bug in my JavaScript code",
    "How do I implement a binary search algorithm?",
    
    # Reasoning queries
    "Why is edge AI becoming important?",
    "Explain step-by-step how neural networks work",
    "Compare the pros and cons of local vs cloud inference",
    
    # Creative queries
    "Write a short story about robots",
    "Create a poem about technology",
    "Write dialogue for a sci-fi movie"
]

test_classification(test_queries)

## লাইভ রাউটিং উদাহরণ

এখন চলুন বাস্তব প্রতিক্রিয়ার সাথে রাউটারকে কার্যকর অবস্থায় দেখি:


In [ ]:
def demonstrate_routing(example_queries: List[str]):
    """Demonstrate live routing with actual model responses."""
    print("🚀 **Live Routing Demonstration**")
    print("=" * 70)
    
    for i, query in enumerate(example_queries, 1):
        print(f"\n{'='*70}")
        print(f"Example {i}: {query}")
        print(f"{'='*70}")
        
        # Route and get response
        result = router.route_and_run(query)
        
        # Display routing information
        print(f"🎯 **Tool Selected:** {result['tool'].title()}")
        print(f"🤖 **Model Used:** {result['model']}")
        print(f"🌡️ **Temperature:** {result['temperature']}")
        print(f"⏱️ **Processing Time:** {result['processing_time']:.2f}s")
        print(f"📝 **Tool Purpose:** {result['tool_description']}")
        
        # Display response
        print(f"\n💬 **Response:**")
        print(result['answer'])

# Example queries for live demonstration
demo_queries = [
    "What are the main benefits of running AI models locally?",
    "Write a Python function to calculate fibonacci numbers",
    "Explain step-by-step why local AI inference is faster than cloud",
    "Write a creative story about an AI assistant living on an edge device"
]

demonstrate_routing(demo_queries)

## পারফরম্যান্স তুলনা

চলুন দেখি কীভাবে বিভিন্ন মডেল একই কাজের উপর পারফর্ম করে:


In [ ]:
def compare_model_performance(test_prompt: str, max_tokens: int = 150):
    """Compare how different models handle the same prompt."""
    print(f"⚖️ **Model Performance Comparison**")
    print(f"📝 **Test Prompt:** {test_prompt}")
    print("=" * 70)
    
    # Test with each configured model
    for tool_name, tool_config in router.tools.items():
        model = tool_config['model']
        temperature = tool_config['temperature']
        
        print(f"\n🔧 **{tool_name.title()} Tool ({model})**")
        print(f"   🌡️ Temperature: {temperature}")
        print("-" * 50)
        
        start_time = time.time()
        response = router.chat(
            model=model, 
            content=test_prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        end_time = time.time()
        
        processing_time = end_time - start_time
        
        print(f"⏱️ **Time:** {processing_time:.2f}s")
        print(f"💬 **Response:** {response}")
        
        # Calculate response length
        response_length = len(response.split())
        print(f"📏 **Length:** {response_length} words")
        
        if "Error" in response:
            print("❌ **Status:** Error")
        else:
            print("✅ **Status:** Success")

# Test with a neutral prompt that could work for any model
comparison_prompt = "Explain the concept of edge computing in simple terms."
compare_model_performance(comparison_prompt)

## ইন্টারঅ্যাকটিভ রাউটার টেস্টিং

আপনার নিজস্ব কাস্টম প্রশ্ন দিয়ে রাউটার পরীক্ষা করুন:


In [ ]:
def interactive_test(custom_query: str, force_tool: str = None):
    """Test the router with a custom query, optionally forcing a specific tool."""
    print(f"🎪 **Interactive Router Test**")
    print(f"💭 **Your Query:** {custom_query}")
    print("=" * 60)
    
    if force_tool:
        if force_tool not in router.tools:
            print(f"❌ Invalid tool: {force_tool}. Available: {list(router.tools.keys())}")
            return
        selected_tool = force_tool
        print(f"🔧 **Forced Tool:** {selected_tool.title()}")
    else:
        selected_tool = router.select_tool(custom_query)
        print(f"🎯 **Auto-Selected Tool:** {selected_tool.title()}")
    
    # Get full result
    if force_tool:
        # Manual routing
        tool_config = router.tools[force_tool]
        model = tool_config['model']
        temperature = tool_config['temperature']
        
        start_time = time.time()
        answer = router.chat(model, custom_query, temperature=temperature)
        end_time = time.time()
        
        result = {
            'tool': force_tool,
            'model': model,
            'temperature': temperature,
            'processing_time': end_time - start_time,
            'answer': answer
        }
    else:
        # Automatic routing
        result = router.route_and_run(custom_query)
    
    # Display results
    print(f"\n📊 **Routing Details:**")
    print(f"   🔧 Tool: {result['tool'].title()}")
    print(f"   🤖 Model: {result['model']}")
    print(f"   🌡️ Temperature: {result['temperature']}")
    print(f"   ⏱️ Time: {result['processing_time']:.2f}s")
    
    print(f"\n💬 **Response:**")
    print(result['answer'])
    
    return result

# Test with your custom query
custom_query = "How can I optimize my Python code for better performance?"
# Uncomment the next line to force a specific tool
# force_tool = "code"  # Options: "general", "reasoning", "code", "creative"
force_tool = None

result = interactive_test(custom_query, force_tool=force_tool)

## উন্নত কনফিগারেশন

রাউটারের কনফিগারেশন কীভাবে কাস্টমাইজ করতে হয় তা প্রদর্শন করুন:


In [ ]:
class CustomModelRouter(ModelRouter):
    """Extended router with custom configuration and additional features."""
    
    def __init__(self, custom_tools: Dict[str, Dict[str, Any]] = None):
        self.custom_tools = custom_tools
        super().__init__()
    
    def _load_tool_registry(self) -> Dict[str, Dict[str, Any]]:
        """Load custom tool registry if provided."""
        if self.custom_tools:
            return self.custom_tools
        return super()._load_tool_registry()
    
    def add_custom_classifier(self, tool_name: str, keywords: List[str]):
        """Add a custom classification rule."""
        self.custom_keywords = getattr(self, 'custom_keywords', {})
        self.custom_keywords[tool_name] = keywords
    
    def select_tool(self, user_query: str) -> str:
        """Enhanced tool selection with custom rules."""
        # Check custom keywords first
        if hasattr(self, 'custom_keywords'):
            query_lower = user_query.lower()
            for tool_name, keywords in self.custom_keywords.items():
                if any(keyword in query_lower for keyword in keywords):
                    return tool_name
        
        # Fall back to default classification
        return super().select_tool(user_query)
    
    def get_tool_statistics(self) -> Dict[str, Any]:
        """Get statistics about tool usage."""
        if not hasattr(self, 'usage_stats'):
            self.usage_stats = {tool: 0 for tool in self.tools.keys()}
            self.total_requests = 0
        
        return {
            'usage_by_tool': self.usage_stats.copy(),
            'total_requests': self.total_requests,
            'most_used_tool': max(self.usage_stats, key=self.usage_stats.get) if self.usage_stats else None
        }
    
    def route_and_run(self, prompt: str) -> Dict[str, Any]:
        """Enhanced routing with usage tracking."""
        result = super().route_and_run(prompt)
        
        # Track usage
        if not hasattr(self, 'usage_stats'):
            self.usage_stats = {tool: 0 for tool in self.tools.keys()}
            self.total_requests = 0
        
        self.usage_stats[result['tool']] += 1
        self.total_requests += 1
        
        return result

# Example: Create a router with custom configuration
custom_config = {
    "general": {
        "model": "phi-4-mini",
        "notes": "General purpose model",
        "temperature": 0.7
    },
    "technical": {
        "model": "qwen2.5-7b-instruct",
        "notes": "Technical documentation and analysis",
        "temperature": 0.3
    },
    "creative": {
        "model": "phi-4-mini",
        "notes": "Creative writing and storytelling",
        "temperature": 0.9
    }
}

print("🔧 **Creating Custom Router Configuration**")
custom_router = CustomModelRouter(custom_tools=custom_config)

# Add custom classification rules
custom_router.add_custom_classifier("technical", ["documentation", "specification", "architecture", "design"])

print("✅ Custom router created with enhanced features")
print(f"🎯 Available tools: {list(custom_router.tools.keys())}")

## ব্যাচ প্রসেসিং উদাহরণ

একাধিক প্রশ্ন দক্ষতার সাথে প্রক্রিয়া করুন:


In [ ]:
def batch_process_queries(queries: List[str], router_instance: ModelRouter = None) -> List[Dict[str, Any]]:
    """Process multiple queries in batch and analyze results."""
    if router_instance is None:
        router_instance = router
    
    print(f"📦 **Batch Processing {len(queries)} Queries**")
    print("=" * 60)
    
    results = []
    total_time = 0
    tool_usage = {}
    
    for i, query in enumerate(queries, 1):
        print(f"\n{i}. Processing: {query[:50]}{'...' if len(query) > 50 else ''}")
        
        result = router_instance.route_and_run(query)
        results.append(result)
        
        # Track statistics
        total_time += result['processing_time']
        tool = result['tool']
        tool_usage[tool] = tool_usage.get(tool, 0) + 1
        
        print(f"   🎯 Tool: {tool.title()} | ⏱️ {result['processing_time']:.2f}s")
    
    # Display summary
    print(f"\n📊 **Batch Processing Summary**")
    print("=" * 40)
    print(f"📝 Total Queries: {len(queries)}")
    print(f"⏱️ Total Time: {total_time:.2f}s")
    print(f"🚀 Average Time: {total_time/len(queries):.2f}s per query")
    
    print(f"\n🎯 **Tool Usage:**")
    for tool, count in sorted(tool_usage.items()):
        percentage = (count / len(queries)) * 100
        print(f"   {tool.title()}: {count} queries ({percentage:.1f}%)")
    
    return results

# Batch test queries
batch_queries = [
    "What is machine learning?",
    "Write a Python function for quicksort",
    "Why is data privacy important in AI?",
    "Create a haiku about artificial intelligence",
    "How do I debug a memory leak in my application?",
    "Explain the difference between supervised and unsupervised learning",
    "Write a short story about a robot chef",
    "What are the advantages of edge computing?"
]

batch_results = batch_process_queries(batch_queries)

# Show detailed results for first few queries
print(f"\n📋 **Sample Detailed Results:**")
for i, result in enumerate(batch_results[:3], 1):
    print(f"\n{i}. **Tool:** {result['tool'].title()} | **Model:** {result['model']}")
    print(f"   **Response:** {result['answer'][:100]}{'...' if len(result['answer']) > 100 else ''}")

## উৎপাদন পর্যবেক্ষণ

উৎপাদনের জন্য প্রস্তুত পর্যবেক্ষণ এবং লগিং-এর উদাহরণ:


In [ ]:
class ProductionModelRouter(ModelRouter):
    """Production-ready router with comprehensive monitoring."""
    
    def __init__(self):
        super().__init__()
        self.metrics = {
            'total_requests': 0,
            'successful_requests': 0,
            'failed_requests': 0,
            'total_processing_time': 0,
            'tool_usage': {},
            'model_performance': {},
            'error_log': []
        }
    
    def route_and_run(self, prompt: str) -> Dict[str, Any]:
        """Enhanced routing with comprehensive monitoring."""
        self.metrics['total_requests'] += 1
        request_id = self.metrics['total_requests']
        
        try:
            result = super().route_and_run(prompt)
            
            # Track success metrics
            self.metrics['successful_requests'] += 1
            self.metrics['total_processing_time'] += result['processing_time']
            
            # Track tool usage
            tool = result['tool']
            if tool not in self.metrics['tool_usage']:
                self.metrics['tool_usage'][tool] = {'count': 0, 'total_time': 0}
            self.metrics['tool_usage'][tool]['count'] += 1
            self.metrics['tool_usage'][tool]['total_time'] += result['processing_time']
            
            # Track model performance
            model = result['model']
            if model not in self.metrics['model_performance']:
                self.metrics['model_performance'][model] = {'requests': 0, 'total_time': 0, 'avg_time': 0}
            self.metrics['model_performance'][model]['requests'] += 1
            self.metrics['model_performance'][model]['total_time'] += result['processing_time']
            self.metrics['model_performance'][model]['avg_time'] = (
                self.metrics['model_performance'][model]['total_time'] / 
                self.metrics['model_performance'][model]['requests']
            )
            
            # Add monitoring metadata
            result['request_id'] = request_id
            result['status'] = 'success'
            
            return result
            
        except Exception as e:
            # Track failure metrics
            self.metrics['failed_requests'] += 1
            error_entry = {
                'request_id': request_id,
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
                'prompt': prompt[:100],  # Truncate for privacy
                'error': str(e)
            }
            self.metrics['error_log'].append(error_entry)
            
            return {
                'request_id': request_id,
                'status': 'error',
                'error': str(e),
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
            }
    
    def get_performance_report(self) -> Dict[str, Any]:
        """Generate comprehensive performance report."""
        total_requests = self.metrics['total_requests']
        if total_requests == 0:
            return {'message': 'No requests processed yet'}
        
        success_rate = (self.metrics['successful_requests'] / total_requests) * 100
        avg_processing_time = (
            self.metrics['total_processing_time'] / max(1, self.metrics['successful_requests'])
        )
        
        return {
            'overview': {
                'total_requests': total_requests,
                'successful_requests': self.metrics['successful_requests'],
                'failed_requests': self.metrics['failed_requests'],
                'success_rate': f"{success_rate:.1f}%",
                'average_processing_time': f"{avg_processing_time:.2f}s"
            },
            'tool_usage': self.metrics['tool_usage'],
            'model_performance': self.metrics['model_performance'],
            'recent_errors': self.metrics['error_log'][-5:] if self.metrics['error_log'] else []
        }

# Create production router and test
print("🏭 **Production Router Testing**")
prod_router = ProductionModelRouter()

# Process several requests
test_requests = [
    "Explain quantum computing",
    "Write a function to reverse a string",
    "Why is cybersecurity important?",
    "Create a poem about the future"
]

print(f"\nProcessing {len(test_requests)} test requests...")
for i, request in enumerate(test_requests, 1):
    result = prod_router.route_and_run(request)
    status = "✅" if result['status'] == 'success' else "❌"
    print(f"{i}. {status} Request {result['request_id']}: {request[:30]}...")

# Generate performance report
report = prod_router.get_performance_report()

print(f"\n📊 **Performance Report**")
print("=" * 50)
print(f"📈 **Overview:**")
for key, value in report['overview'].items():
    print(f"   {key.replace('_', ' ').title()}: {value}")

print(f"\n🎯 **Tool Usage:**")
for tool, stats in report['tool_usage'].items():
    avg_time = stats['total_time'] / stats['count']
    print(f"   {tool.title()}: {stats['count']} requests (avg: {avg_time:.2f}s)")

print(f"\n🤖 **Model Performance:**")
for model, stats in report['model_performance'].items():
    print(f"   {model}: {stats['requests']} requests (avg: {stats['avg_time']:.2f}s)")

if report['recent_errors']:
    print(f"\n❌ **Recent Errors:**")
    for error in report['recent_errors']:
        print(f"   Request {error['request_id']}: {error['error']}")
else:
    print(f"\n✅ **No Recent Errors**")

## সারাংশ এবং সেরা অনুশীলন

এই নোটবুকে একটি উন্নত বুদ্ধিমান মডেল রাউটিং সিস্টেম প্রদর্শিত হয়েছে:

### ✅ প্রদর্শিত প্রধান বৈশিষ্ট্যসমূহ

1. **🎯 বুদ্ধিমান শ্রেণীবিভাগ**: কীওয়ার্ড বিশ্লেষণের মাধ্যমে স্বয়ংক্রিয় কাজ শ্রেণীকরণ
2. **🔧 গতিশীল মডেল নির্বাচন**: বিশেষায়িত মডেলের দিকে প্রশ্ন রাউটিং
3. **⚙️ নমনীয় কনফিগারেশন**: পরিবেশ-ভিত্তিক এবং কাস্টম টুল রেজিস্ট্রি
4. **📊 স্বাস্থ্য পর্যবেক্ষণ**: পরিষেবা এবং মডেলের প্রাপ্যতা পরীক্ষা
5. **⚡ কর্মক্ষমতা ট্র্যাকিং**: প্রতিক্রিয়া সময় এবং ব্যবহার বিশ্লেষণ
6. **🏭 উৎপাদন প্রস্তুত**: ব্যাপক পর্যবেক্ষণ এবং ত্রুটি পরিচালনা

### 🎨 মডেল ক্যাটাগরি সারাংশ

| ক্যাটাগরি | ডিফল্ট মডেল | টেম্পারেচার | সেরা ব্যবহারের ক্ষেত্র |
|-----------|-------------|-------------|-------------------------|
| **🌐 সাধারণ** | phi-4-mini | 0.7 | প্রশ্নোত্তর, চ্যাট, সাধারণ কাজ |
| **🧠 যুক্তি** | deepseek-r1-distill-qwen-7b | 0.3 | বিশ্লেষণ, ব্যাখ্যা |
| **💻 কোড** | qwen2.5-7b-instruct | 0.2 | প্রোগ্রামিং, ডিবাগিং |
| **🎨 সৃজনশীল** | phi-4-mini | 0.9 | গল্প, কবিতা, সৃজনশীল লেখা |

### 🔍 শ্রেণীবিভাগের নিয়ম

- **কোড শনাক্তকরণ**: `code`, `python`, `function`, `class`, `bug`, `debug`, `programming`
- **যুক্তি শনাক্তকরণ**: `why`, `how`, `explain`, `step-by-step`, `reason`, `analyze`
- **সৃজনশীল শনাক্তকরণ**: `story`, `poem`, `creative`, `imagine`, `write`, `tale`
- **ডিফল্ট**: অন্যান্য সমস্ত প্রশ্ন সাধারণ মডেল ব্যবহার করে

### 💡 সেরা অনুশীলন

1. **🎯 মডেল বিশেষায়ন**: বিভিন্ন মডেলের শক্তি অনুযায়ী ব্যবহার করুন
2. **🌡️ টেম্পারেচার টিউনিং**: নির্ভুলতার জন্য কম, সৃজনশীলতার জন্য বেশি
3. **📊 ক্রমাগত পর্যবেক্ষণ**: কর্মক্ষমতা এবং ব্যবহার প্যাটার্ন ট্র্যাক করুন
4. **🔄 ব্যাকআপ কৌশল**: মডেল অনুপলব্ধ হলে গ্রেসফুল ডিগ্রেডেশন
5. **⚖️ লোড ব্যালেন্সিং**: একাধিক ইনস্ট্যান্সে লোড বিতরণ করুন
6. **🔧 কাস্টম শ্রেণীবিভাগ**: ডোমেইন-নির্দিষ্ট রাউটিং নিয়ম যোগ করুন

### 🚀 ব্যবহার ক্ষেত্র

- **👨‍💻 ডেভেলপমেন্ট টুলস**: বুদ্ধিমান মডেল রাউটিং সহ IDE প্লাগইন
- **🎧 গ্রাহক সহায়তা**: বিশেষায়িত সহায়তা মডেলের দিকে প্রশ্ন রাউটিং
- **📝 বিষয়বস্তু তৈরি**: উপযুক্ত সৃজনশীল মডেলের সাথে কাজ মেলানো
- **📚 ডকুমেন্টেশন**: প্রযুক্তিগত লেখার মডেলের দিকে রাউটিং
- **🎓 শিক্ষামূলক প্ল্যাটফর্ম**: বিষয়-নির্দিষ্ট মডেল রাউটিং

### 🔮 পরবর্তী পদক্ষেপ

- **🤖 মেশিন লার্নিং**: আরও ভালো শ্রেণীবিভাগের জন্য ML ব্যবহার
- **🔧 ফাংশন কলিং**: নির্দিষ্ট টুল ক্ষমতা সহ মডেলের দিকে রাউটিং
- **📸 মাল্টি-মোডাল**: ইনপুট টাইপ (টেক্সট, ইমেজ, অডিও) অনুযায়ী রাউটিং
- **🌐 ফেডারেটেড রাউটিং**: একাধিক Foundry Local ইনস্ট্যান্সের মধ্যে রাউটিং
- **💰 খরচ অপ্টিমাইজেশন**: খরচ এবং কর্মক্ষমতা মেট্রিকের উপর ভিত্তি করে রাউটিং

এই বুদ্ধিমান রাউটিং সিস্টেমটি দেখায় কীভাবে স্থানীয় ইনফারেন্সের গোপনীয়তা এবং কর্মক্ষমতার সুবিধা বজায় রেখে একাধিক বিশেষায়িত মডেলের মূল্য সর্বাধিক করা যায়। "Models as Tools" প্যারাডাইমটি এমন উন্নত AI অ্যাপ্লিকেশন তৈরি করতে সক্ষম করে যা প্রতিটি নির্দিষ্ট কাজের জন্য সেরা মডেলটি স্বয়ংক্রিয়ভাবে নির্বাচন করে।
